# QC for FLICA project
Initial filtering for FLICA:
- Resting state dicom instance 2 not null
- Multiband diffusion dicom instance 2 not null

Imaging qc:
- registration failure in TBM or dMRI (from Sourena)
- resting state or diffusion qc metrics (see column names below) outside 5 standard deviations median
    
Genomic qc:
- Genetic sex == reported sex
- No sex chromosome aneuploidy
- Kinship
- Ancestry
- Variant QC performed later
    - missingness
    - HWE

Outputs:
- subject list that passed QC
- covariates genetic analyses dataframe
- behavioural phenotypes dataframe


TODO:
- Finish genetic QC with subject list from Barbara
- Make maybe list
- Finish covariates
- Finish exports

Last edit: 2023-11-21, J.S. Amelink

## 0. Set up

In [1]:
import pandas as pd
import os
import glob
import scipy as sp
import numpy as np

workspace_path = "/data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/"
cfs_path = "/data/clusterfs/lag/projects/lg-ukbiobank/working_data/imaging_data/FLICA_multimodal/"

#number of standard deviations for imaging QC
no_std = 3.5

#columns to do exclusion on:
qc_cols = [
    #T1
    'Amount of warping applied to non-linearly align T1 brain image to standard-space | Instance 2',
    'Inverted signal-to-noise ratio in T1 | Instance 2',
    
    #Resting state
    'Inverted temporal signal-to-noise ratio in artefact-cleaned pre-processed rfMRI | Instance 2',
    'Mean rfMRI head motion averaged across space and time points | Instance 2',
    
    #Diffusion
    'Number of dMRI outlier slices detected and corrected | Instance 2',
    'Standard deviation of apparent translation in the Y axis as measured by eddy | Instance 2',
 ]

In [2]:
#load data and adapt labels
df = pd.read_csv(os.path.join(workspace_path, 'CONGRADS_FLICA_participant.tsv'), sep="\t", engine="pyarrow", dtype={"eid": np.int64})

rename_dict = {"eid" : 'Participant ID',
               "p22001" : "Genetic sex",
               "p21003_i2" : 'Age when attended assessment centre | Instance 2',
               "p25733_i2" : 'Amount of warping applied to non-linearly align T1 brain image to standard-space | Instance 2',
               "p25756_i2" : 'Scanner lateral (X) brain position | Instance 2',
               "p25757_i2" : 'Scanner transverse (Y) brain position | Instance 2',
               "p25758_i2" : 'Scanner longitudinal (Z) brain position | Instance 2',
               "p25744_i2" : 'Inverted temporal signal-to-noise ratio in artefact-cleaned pre-processed rfMRI | Instance 2',
               "p25741_i2" : 'Mean rfMRI head motion averaged across space and time points | Instance 2',
               "p25737_i2" : 'Discrepancy between dMRI brain image and T1 brain image | Instance 2',
               "p25739_i2" : 'Discrepancy between rfMRI brain image and T1 brain image | Instance 2',
               "p25922_i2" : 'Standard deviation of apparent translation in the Y axis as measured by eddy | Instance 2',
               "p25746_i2" : 'Number of dMRI outlier slices detected and corrected | Instance 2',
               "p22000" : 'Genotype measurement batch',
               "p32050" : 'Exome release tranche',
               "p22009_a1" :'Genetic principal components | Array 1',
               "p22009_a2" : 'Genetic principal components | Array 2',
               "p22009_a3" :'Genetic principal components | Array 3',
               "p22009_a4" : 'Genetic principal components | Array 4',
               "p22009_a5" : 'Genetic principal components | Array 5',
               "p22009_a6" : 'Genetic principal components | Array 6',
               "p22009_a7" : 'Genetic principal components | Array 7',
               "p22009_a8" : 'Genetic principal components | Array 8',
               "p22009_a9" : 'Genetic principal components | Array 9',
               "p22009_a10" : 'Genetic principal components | Array 10',
               "p54_i2" : "UK Biobank assessment centre | Instance 2",
               "p1707_i2" : 'Handedness (chirality/laterality) | Instance 2',
               "p1707_i0" : 'Handedness (chirality/laterality) | Instance 0',
               "p41270" : 'Diagnoses - ICD10',
               "p41204" : 'Diagnoses - secondary ICD10',
               "p20016_i2" : "Fluid intelligence score | Instance 2",
               "p22019" : "Sex chromosome aneuploidy",
               "p20252_i2" : "T1 structural brain images - NIFTI | Instance 2",
               "p20250_i2" : "Multiband diffusion brain images - NIFTI | Instance 2",
               "p20227_i2" : "Functional brain images - resting - NIFTI | Instance 2",
               "p25734_i2" : "Inverted signal-to-noise ratio in T1 | Instance 2",
               "p31" : "Sex" }

#column_labels = pd.read_csv(os.path.join(workspace_path, "FLICA_table_v2_lables_participant.tsv"), sep="\t")
#rename_dict = dict(zip(list(df.columns), list(column_labels)))
df.rename(columns=rename_dict, inplace=True)

## 1. Imaging QC

In [3]:
#get participants that pass genetic QC Sourena
sourena_qc = pd.read_csv(os.path.join(workspace_path, "qc-sourena", "c0-post-QC"), sep=" ", header=None, names=["sid", "qc_pass"])

#exclusion
exclusion_file_dmri_sourena = open(os.path.join(workspace_path, "qc-sourena", "remove-list-dMRI-all-sorted.txt"))
exclusion_list_dmri = [int(x) for x in sorted(list(exclusion_file_dmri_sourena.read().split(' remove\n')))[1:]]
exclusion_file_tbm_sourena = open(os.path.join(workspace_path, "qc-sourena", "remove-list-tbm-all-sorted.txt"))
exclusion_list_tbm = [int(x) for x in sorted(list(exclusion_file_tbm_sourena.read().split(' remove\n')))[1:]]

all_t1s_sourena = pd.read_csv(os.path.join(workspace_path, "qc-sourena","list-t1-all.txt"),
                              names=["Participant ID", "nifti_rain", "unbiased_brain", "brain_mask"],
                              dtype={"Participant ID": np.int64},
                              sep=" ")

all_t1_sou_set = set(all_t1s_sourena["Participant ID"]) 

#get imaging QC vs genetic qc
subs_qc = set(sourena_qc["sid"])
subs_qc_fail_gen = set(sourena_qc.loc[sourena_qc.qc_pass == "remove", "sid"])
subs_qc_fail_dmri = set(exclusion_list_dmri) #- subs_qc_fail_gen
subs_qc_fail_tbm = set(exclusion_list_tbm) #- subs_qc_fail_gen

qc_missing_sou = np.bool_([ True if x not in subs_qc else False for x in df["Participant ID"] ]).reshape((1, len(df)))
qc_fail_dmri = np.bool_([ True if x in subs_qc_fail_dmri else False for x in df["Participant ID"] ]).reshape((1, len(df)))
qc_fail_tbm = np.bool_([ True if x in subs_qc_fail_tbm else False for x in df["Participant ID"] ]).reshape((1, len(df)))
qc_not_in_sou = np.bool_([ True if x not in all_t1_sou_set else False for x in df["Participant ID"] ]).reshape((1, len(df)))
qc_fail_gen = np.bool_([ True if x not in subs_qc_fail_gen else False for x in df["Participant ID"] ]).reshape((1, len(df)))

qc_fail_sou = np.logical_or(qc_fail_dmri, qc_fail_tbm)

qc_exclude = np.logical_or(qc_fail_tbm, qc_not_in_sou)

#print("dMRI QC fail Sourena (no genetics): {}".format(np.sum(qc_fail_dmri)))
print("TBM QC fail Sourena (no genetics): {}".format(np.sum(qc_fail_tbm)))
print("Fail either or both: {}".format(np.sum(qc_exclude)))
print("Subjects not in imaging QC Sourena: {}".format(np.sum(qc_not_in_sou)))

TBM QC fail Sourena (no genetics): 1096
Fail either or both: 5277
Subjects not in imaging QC Sourena: 4191


In [4]:
#get participants with data on cluster
rs_data_zip = list(glob.glob("/data/workspaces/lag/workspaces/lg-ukbiobank/primary_data/imaging_data/*/*_20227_2_0.zip"))
diff_data_zip = list(glob.glob("/data/workspaces/lag/workspaces/lg-ukbiobank/primary_data/imaging_data/*/*_20250_2_0.zip"))
rs_part_zip = [x[-21:-14] for x in rs_data_zip]
diff_part_zip = [x[-21:-14] for x in diff_data_zip]

print("Number of subjects with diffusion data on the cluster: {}".format(len(diff_data_zip)))
print("Number of subjects with resting state data on the cluster: {}".format(len(rs_data_zip)))

overlap = []
diff_part_set = set(diff_part_zip)
for sub in rs_part_zip:
    if sub in diff_part_set:
        overlap.append(sub)
    
print("Number of subjects with both data types available: {}".format(len(overlap)))
imaging_cluster = np.bool_( [True if x in overlap else False for x in df["Participant ID"] ] ).reshape((1, len(df)))

Number of subjects with diffusion data on the cluster: 47557
Number of subjects with resting state data on the cluster: 49198
Number of subjects with both data types available: 47514


In [5]:
#check missing data
no_part_start = len(df)
print("Number of participants with in dataframe: {}".format(no_part_start))

qc_missing_rs = np.bool_(df['Mean rfMRI head motion averaged across space and time points | Instance 2'].isna()).reshape((1, len(df)))
qc_missing_diff = np.bool_(df[ 'Number of dMRI outlier slices detected and corrected | Instance 2'].isna()).reshape((1, len(df)))
qc_missing = np.logical_or(qc_missing_rs, qc_missing_diff, qc_missing_sou)

print("Subjects without resting state imaging qc metrics: {}".format(np.sum(qc_missing_rs)))
print("Subjects without diffusion imaging qc metrics: {}".format(np.sum(qc_missing_diff)))
print("Subjects without one or more imaging qc metrics: {}".format(np.sum(qc_missing)))

Number of participants with in dataframe: 41584
Subjects without resting state imaging qc metrics: 3905
Subjects without diffusion imaging qc metrics: 4299
Subjects without one or more imaging qc metrics: 4355


In [6]:
#imaging qc:
stds=df[qc_cols].std()
means=df[qc_cols].mean()

high_cut_off = means+no_std*stds
high = np.bool_([np.array(df[qc_cols] < high_cut_off).sum(axis=1) < len(qc_cols)]).reshape((1, len(df)))

high_excl = set(df.loc[high.T, "Participant ID"]) - set(df.loc[qc_missing_rs.T, "Participant ID"])

qc_exclude = np.logical_or(high, qc_fail_tbm)
#qc_missing = np.logical_or(qc_mqc_not_in_sou)
##imaging_qc_all = np.logical_or(qc_exclude, qc_missing)
overlap_tbm_high = np.logical_and(high, qc_fail_tbm)
overlap_tbm_missing_rs = np.logical_and(qc_fail_tbm, qc_missing_rs)
overlap_tbm_missing_dti = np.logical_and(qc_fail_tbm, qc_missing_diff)
overlap_missing_dti_rs = np.logical_and(qc_missing_diff, qc_missing_rs)

print("Subjects that failed imaging QC Sourena: {}".format(np.sum(qc_fail_tbm)))
print("Subjects outside {0} standard deviations: {1}".format( no_std, len(high_excl)) )
print("Subjects that failed metric QC outside {0} standard deviations (includes missing QC metrics): {1}".format(no_std, np.sum(high)))
print("Subjects with resting QC missing: {}".format(np.sum(qc_missing_rs)))
print("Subjects not in imaging QC Sourena: {}".format(np.sum(qc_not_in_sou)))
print("Subjects overlap imaging QC Sourena && 3 std: {}".format(np.sum(overlap_tbm_high)))
print("Subjects overlap imaging QC Sourena && unusable fMRI: {}".format(np.sum(overlap_tbm_missing_rs)))
print("Subjects overlap imaging QC Sourena && unusable dMRI: {}".format(np.sum(overlap_tbm_missing_dti)))
print("Subjects overlap imaging unusable dMRI && fMRI: {}".format(np.sum(overlap_tbm_missing_dti)))
#print("Subjects that have no QC status available: {}".format(np.sum(qc_missing)))
print("Number of subjects excluded after imaging QC: {}".format(np.sum(qc_exclude)))

Subjects that failed imaging QC Sourena: 1096
Subjects outside 3.5 standard deviations: 2133
Subjects that failed metric QC outside 3.5 standard deviations (includes missing QC metrics): 6038
Subjects with resting QC missing: 3905
Subjects not in imaging QC Sourena: 4191
Subjects overlap imaging QC Sourena && 3 std: 94
Subjects overlap imaging QC Sourena && unusable fMRI: 40
Subjects overlap imaging QC Sourena && unusable dMRI: 44
Subjects overlap imaging unusable dMRI && fMRI: 44
Number of subjects excluded after imaging QC: 7040


## 2. Genetic QC

In [9]:
#sex check
exome_list_file = open("/data/workspaces/lag/workspaces/lg-ukbiobank/derived_data/genetic_data/exome/exome_release_450k/sex_check/ukb_exome_450k_white_british_ancestry_individuals_discordant_reported_genetic_sex_female_0.3_male_0.7.txt")
exome_sex_check = set([int(x) for x in sorted(list(exome_list_file.read().split('\n')))[1:]])

#exome_batch = pd.read_csv("/data/workspaces/lag/workspaces/lg-ukbiobank/derived_data/genetic_data/exome/exome_release_final/exome_batch/exome_batches.txt",
#                          sep="\t",
#                          names=["Participant ID", "exome_batch"],
#                          dtype={"Participant ID": np.int64, "exome_batch": np.int32},
#                          skiprows=1)
#df = df.join(exome_batch['exome_batch'], how='left', on="Participant ID")

#gwas_check_wb_file = open("/data/workspaces/lag/workspaces/lg-ukbiobank/derived_data/genetic_data/exome/exome_release_450k/sex_check/ukb_exome_450k_white_british_ancestry_individuals_discordant_reported_genetic_sex_female_0.3_male_0.7.txt")
#gwas_check_list = [int(x) for x in sorted(list(gwas_check_wb_file.read().split('\n')))[1:]]

gwas_check_wb_file = open("/data/workspaces/lag/workspaces/lg-ukbiobank/derived_data/genetic_data/snp/subset_imagingT1_50k/whiteBritish_subset/imagingT1_exclude_POSTrelatedness.txt")
gwas_check_list = [int(x) for x in sorted(list(gwas_check_wb_file.read().split('\n')))[1:]]

sex_chr_aneuploidy = ~df["Sex chromosome aneuploidy"].isna()
genetic_sex = np.bool_(df["Sex"] != df["Genetic sex"])
exome_check = np.bool_( [True if x in exome_sex_check else False for x in df["Participant ID"] ] )
gwas_check = np.bool_( [True if x in gwas_check_list else False for x in df["Participant ID"] ] )
sex_check = np.logical_or(sex_chr_aneuploidy, genetic_sex)
gen_check = np.logical_or(gwas_check, exome_check, sex_check) 

basic_filter = np.logical_or(qc_exclude, gen_check)
gen_img_both = np.logical_and(qc_exclude, gen_check)

print("Number of subjects excluded with sex check (GWAS and exome), sex chromosome aneuploidy or genetic relatedness: {}".format(np.sum(gen_check)))
print("Number of subjects excluded with sex check (GWAS and exome), sex chromosome aneuploidy or genetic relatedness after imaging QC: {}".format(np.sum(gen_check) - np.sum(gen_img_both)))
print("Total number of subjects excluded: {}".format(np.sum(basic_filter)))

Number of subjects excluded with sex check (GWAS and exome), sex chromosome aneuploidy or genetic relatedness: 1399
Number of subjects excluded with sex check (GWAS and exome), sex chromosome aneuploidy or genetic relatedness after imaging QC: 625
Total number of subjects excluded: 8828


## 3. Exporting:
- include list
- exclude list
- missing list (maybe second round)
- filtered dataframe
- behavioural 
- covariates

In [10]:
#TODO ADD:
#FILTER MISSING LIST FOR GENETIC QC AND SOURENA'S IMAGING QC -> maybe list

#participant lists
include_list = df.loc[~basic_filter.T, "Participant ID"].to_csv(os.path.join(workspace_path,"participant_list_imaging_sex_check_50k.txt"),index=False,header=False, sep="\t") 

#maybe_list = df.loc[qc_missing.T, "Participant ID"].to_csv(os.path.join(workspace_path,"maybe_participants_new.txt"),index=False,header=False, sep="\t")

keep = df.loc[~basic_filter.T, :]
keep.to_csv(os.path.join(workspace_path,"FLICA_filtered_data_50k.tsv"), index=False, sep="\t")

print("Number of participants included: {}".format(keep.shape))

Number of participants included: (32756, 37)


In [ ]:
! cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/participant_list_imaging_sex_check.txt | head -n 100 > /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/participants_hundred.txt
! cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/participant_list_imaging_sex_check.txt | head -n 1000 | tail -n 900 > /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/participants_thousand.txt 
! cat /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/participant_list_imaging_sex_check.txt | head -n 20000 | tail -n 10000 > /data/workspaces/lag/workspaces/lg-ukbiobank/projects/FLICA_multimodal/participants_next_ten_thousand.txt 

In [13]:
#save
behavior = ['Participant ID',
            'Sex',
            #'Age when attended assessment centre | Instance 0',
            'Age when attended assessment centre | Instance 2',
            'Handedness (chirality/laterality) | Instance 0',
            'Handedness (chirality/laterality) | Instance 2',
            'Diagnoses - ICD10',
            #'Diagnoses - main ICD10',
            'Diagnoses - secondary ICD10',
            ]

keep[behavior].to_csv(os.path.join(workspace_path, "behavioural_scores_N{}.tsv".format(len(keep))), sep="\t", index=False)

In [14]:
#covariates

covariates = [ #subject characteristics
              'Participant ID',
              'Genetic sex',
              'Age when attended assessment centre | Instance 2',
    
                #T1
              'Amount of warping applied to non-linearly align T1 brain image to standard-space | Instance 2',
              'Scanner lateral (X) brain position | Instance 2',
              'Scanner transverse (Y) brain position | Instance 2',
              'Scanner longitudinal (Z) brain position | Instance 2',
    
                #rfMRI
              'Inverted temporal signal-to-noise ratio in artefact-cleaned pre-processed rfMRI | Instance 2',
              'Mean rfMRI head motion averaged across space and time points | Instance 2',
              'Discrepancy between rfMRI brain image and T1 brain image | Instance 2',
    
                #dMRI
              'Discrepancy between dMRI brain image and T1 brain image | Instance 2',
              'Standard deviation of apparent translation in the Y axis as measured by eddy | Instance 2',
              'Number of dMRI outlier slices detected and corrected | Instance 2',
    
                #genetic
              'Genetic principal components | Array 1',
              'Genetic principal components | Array 2',
              'Genetic principal components | Array 3',
              'Genetic principal components | Array 4',
              'Genetic principal components | Array 5',
              'Genetic principal components | Array 6',
              'Genetic principal components | Array 7',
              'Genetic principal components | Array 8',
              'Genetic principal components | Array 9',
              'Genetic principal components | Array 10'
]

covs = keep[covariates]

covs['age_sq'] = np.square(covs['Age when attended assessment centre | Instance 2'])
covs['age_sex'] = covs['Age when attended assessment centre | Instance 2']*covs['Genetic sex']

#make dummies
covs['geno_array_dummy'] = 1
covs.loc[keep['Genotype measurement batch'] < 0, 'geno_array_dummy'] = 0

site_dummies = pd.get_dummies(keep['UK Biobank assessment centre | Instance 2'])
site_dummies.columns = ["site_dummy_{0}".format(x) for x in site_dummies.columns]

covs['exome_dummy'] = 1
covs.loc[keep["Exome release tranche"] > 3.1, 'exome_dummy'] = 0

covs = pd.concat([covs, site_dummies], axis=1)
covs.columns = [x.replace(" ","_") for x in covs.columns]

covs = covs[ ['Participant_ID'] + list(covs.columns[0:]) ]

covs.columns = [ ['FID', 'IID'] + list(covs.columns[2:]) ]

covs.to_csv(os.path.join(workspace_path, "regenie_covariates_50k.tsv"), sep="\t", index=False)

/tmp/ipykernel_50634/2692371217.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covs['age_sq'] = np.square(covs['Age when attended assessment centre | Instance 2'])
/tmp/ipykernel_50634/2692371217.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  covs['age_sex'] = covs['Age when attended assessment centre | Instance 2']*covs['Genetic sex']
/tmp/ipykernel_50634/2692371217.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_